<a href="https://colab.research.google.com/github/Meta-Sean/sqzme/blob/main/ftd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import sys
import shutil
import requests
import pandas as pd
from datetime import timedelta
from pathlib import Path
from bs4 import BeautifulSoup
easy = []
def download_ftd():

  base_url = "https://www.sec.gov"
  text_soup_high_short_interested_stocks = BeautifulSoup(
  requests.get("https://www.sec.gov/data/foiadocsfailsdatahtm").text, "lxml")

  ftd_url_links = text_soup_high_short_interested_stocks.findAll("table")[1].findAll("a")
  for url_link in ftd_url_links[:24]:
    print(base_url + url_link["href"])
    df = pd.read_csv(base_url + url_link["href"], delimiter="|", error_bad_lines=False, encoding="cp1252")
    easy.append(url_link["href"].split("/")[-1].replace(".zip", ".csv"))
    df.to_csv(url_link["href"].split("/")[-1].replace(".zip", ".csv"),
                  index=None)

download_ftd()
      

https://www.sec.gov/files/data/fails-deliver-data/cnsfails202207b.zip


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.




https://www.sec.gov/files/data/fails-deliver-data/cnsfails202207a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202206b.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202206a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202205b.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202205a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202204b.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202204a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202203b.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202203a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202202b.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202202a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202201b.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202201a.zip
https://www.sec.gov/files/data/fails-deliver-data/cnsfails202112b.zip
https://www.sec.gov/

In [17]:
def upload_to_df(files_available, limit=24):
    """
    Combine all the 1/2 monthly csv into 1 large csv file
    """
    combined_df = pd.DataFrame(columns=["SETTLEMENT DATE", "SYMBOL", "QUANTITY (FAILS)", "PRICE"])
    for file in files_available[:limit]:
        print("Processing: ", file)
        df = pd.read_csv(file)
        del df["CUSIP"]
        del df["DESCRIPTION"]
        df["SETTLEMENT DATE"] = df["SETTLEMENT DATE"].apply(lambda x: str(x[:4] + "-" + x[4:6] + "-" + x[6:]))
        combined_df = combined_df.append(df).drop_duplicates()

    combined_df["SETTLEMENT DATE"] = combined_df["SETTLEMENT DATE"].astype(str)
    combined_df.rename(columns={"SETTLEMENT DATE": "Date",
                                "SYMBOL": "Ticker",
                                "QUANTITY (FAILS)": "Failure to Deliver",
                                "PRICE": "Price"}, inplace=True)
    combined_df["T+35 Date"] = pd.to_datetime(combined_df['Date'], format='%Y-%m-%d', errors="coerce") + timedelta(days=35)
    combined_df["T+35 Date"] = combined_df["T+35 Date"].astype(str)
    combined_df = combined_df[~combined_df["Ticker"].isna()]
    combined_df.sort_values(by="Date", inplace=True)
    print(combined_df)

upload_to_df(easy, limit=1)


Processing:  cnsfails202207b.csv
             Date Ticker  Failure to Deliver   Price   T+35 Date
0      2022-07-15   EURN            187268.0   12.00  2022-08-19
3847   2022-07-15   NTCT                 1.0   31.53  2022-08-19
3846   2022-07-15   NTWK                61.0    3.08  2022-08-19
3845   2022-07-15   NCPL              7707.0    2.85  2022-08-19
3844   2022-07-15   NTES               972.0   87.94  2022-08-19
...           ...    ...                 ...     ...         ...
59481  2022-07-29    EFX               854.0  207.50  2022-09-02
59480  2022-07-29  EPHYW              2818.0    0.09  2022-09-02
59479  2022-07-29  EPOKY                47.0   16.95  2022-09-02
59477  2022-07-29  EOSEW              1395.0    0.48  2022-09-02
63355  2022-07-29     ZY              2176.0    2.45  2022-09-02

[63349 rows x 5 columns]


In [21]:
def get_top_ftd(filename):
    """
    Get top stocks with high/consistent FTD.
    Criteria: more than 1 days of >1000000 FTD in 2 weeks
    """
    print("Getting top FTD for: ", filename)
    df = pd.read_csv(filename)
    df.sort_values(by=["SYMBOL", "SETTLEMENT DATE"], ascending=False, inplace=True)
    df["PRICE"] = df["PRICE"].apply(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0)

    original_df = df.copy()

    # Criteria
    df = df[df["QUANTITY (FAILS)"] >= 1000000]
    df = df[df["PRICE"] >= 1]
    df = df.groupby("SYMBOL").filter(lambda x: len(x) >= 1)

    # We want to extract all ftd quantities from original df
    original_df = original_df[original_df["SYMBOL"].isin(df["SYMBOL"].unique())]

    del original_df["CUSIP"]
    del original_df["DESCRIPTION"]

    original_df["SETTLEMENT DATE"] = original_df["SETTLEMENT DATE"].apply(lambda x: str(x[:4] + "-" + x[4:6] + "-" + x[6:]))
    original_df["SETTLEMENT DATE"] = original_df["SETTLEMENT DATE"].astype(str)
    original_df["FTD x $"] = (original_df["QUANTITY (FAILS)"].astype(int) * original_df["PRICE"].astype(float)).astype(int)
    original_df.rename(columns={"SETTLEMENT DATE": "Date",
                                "SYMBOL": "Ticker",
                                "QUANTITY (FAILS)": "FTD",
                                "PRICE": "Price"}, inplace=True)
    original_df["T+35 Date"] = pd.to_datetime(original_df['Date'], format='%Y-%m-%d', errors="coerce") + timedelta(
        days=35)
    original_df["T+35 Date"] = original_df["T+35 Date"].astype(str)  # .apply(lambda x: x.replace("-", "/"))

    # Sort df based on number of FTD days that meet criteria and add new row between tickers
    combined_df = pd.DataFrame(columns=["Date", "Ticker", "FTD", "Price", "FTD x $"])
    for i in df["SYMBOL"].value_counts().index:
        individual_df = original_df[original_df["Ticker"] == i]
        individual_df = individual_df.append([""])
        combined_df = combined_df.append(individual_df)
    del combined_df[0]
    return combined_df

In [41]:
df = get_top_ftd(easy[0])
df

Getting top FTD for:  cnsfails202207b.csv


,Date,Ticker,FTD,Price,FTD x $,T+35 Date
59575,2022-07-29,FFIE,1922349.0,2.21,4248391.0,2022-09-02
53781,2022-07-28,FFIE,1819582.0,2.39,4348800.0,2022-09-01
47967,2022-07-27,FFIE,1447347.0,2.15,3111796.0,2022-08-31
42215,2022-07-26,FFIE,988650.0,2.30,2273895.0,2022-08-30
36503,2022-07-25,FFIE,4442522.0,2.60,11550557.0,2022-08-29
...,...,...,...,...,...,...
49127,2022-07-27,IXUS,560068.0,57.09,31974282.0,2022-08-31
43351,2022-07-26,IXUS,382972.0,57.82,22143441.0,2022-08-30
37634,2022-07-25,IXUS,578762.0,57.42,33232514.0,2022-08-29
31898,2022-07-22,IXUS,1080704.0,57.72,62378234.0,2022-08-26


In [42]:
# Take a user ticker and output the relevant row 
ticker = input("What ticker should we look up? ")
dfs = (df.index[df['Ticker']==ticker].tolist())
dfs

What ticker should we look up? REV


,Date,Ticker,FTD,Price,FTD x $,T+35 Date
59575,2022-07-29,FFIE,1922349.0,2.21,4248391.0,2022-09-02
53781,2022-07-28,FFIE,1819582.0,2.39,4348800.0,2022-09-01
47967,2022-07-27,FFIE,1447347.0,2.15,3111796.0,2022-08-31
42215,2022-07-26,FFIE,988650.0,2.30,2273895.0,2022-08-30
36503,2022-07-25,FFIE,4442522.0,2.60,11550557.0,2022-08-29
...,...,...,...,...,...,...
49127,2022-07-27,IXUS,560068.0,57.09,31974282.0,2022-08-31
43351,2022-07-26,IXUS,382972.0,57.82,22143441.0,2022-08-30
37634,2022-07-25,IXUS,578762.0,57.42,33232514.0,2022-08-29
31898,2022-07-22,IXUS,1080704.0,57.72,62378234.0,2022-08-26


In [48]:
df_dict = {}
for index in dfs:
  df_dict['Date'] = df['Date'][dfs]
  df_dict['FTD'] = df['FTD'][dfs]
  df_dict['Price'] = df['Price'][dfs]
  df_dict['FTD x $'] = df['FTD x $'][dfs]
  df_dict['T+35 Date'] = df['T+35 Date'][dfs]


In [49]:
df_dict

{'Date': 62146    2022-07-29
 56332    2022-07-28
 50531    2022-07-27
 44731    2022-07-26
 39003    2022-07-25
 33253    2022-07-22
 27516    2022-07-21
 21834    2022-07-20
 15979    2022-07-19
 10205    2022-07-18
 4565     2022-07-15
 Name: Date, dtype: object, 'FTD': 62146     171242.0
 56332     125703.0
 50531     121294.0
 44731     367279.0
 39003     359172.0
 33253     651017.0
 27516     769571.0
 21834    1045968.0
 15979    1532108.0
 10205    1492163.0
 4565     1371224.0
 Name: FTD, dtype: float64, 'Price': 62146    4.77
 56332    4.76
 50531    4.96
 44731    4.94
 39003    5.45
 33253    5.62
 27516    5.43
 21834    5.63
 15979    5.49
 10205    5.75
 4565     5.81
 Name: Price, dtype: float64, 'FTD x $': 62146     816824.0
 56332     598346.0
 50531     601618.0
 44731    1814358.0
 39003    1957487.0
 33253    3658715.0
 27516    4178770.0
 21834    5888799.0
 15979    8411272.0
 10205    8579937.0
 4565     7966811.0
 Name: FTD x $, dtype: float64, 'T+35 Date': 6

In [50]:
ticker_df = pd.DataFrame(df_dict)
ticker_df = ticker_df.set_index('Date')

In [51]:
ticker_df

,FTD,Price,FTD x $,T+35 Date
Date,,,,
2022-07-29,171242.0,4.77,816824.0,2022-09-02
2022-07-28,125703.0,4.76,598346.0,2022-09-01
2022-07-27,121294.0,4.96,601618.0,2022-08-31
2022-07-26,367279.0,4.94,1814358.0,2022-08-30
2022-07-25,359172.0,5.45,1957487.0,2022-08-29
2022-07-22,651017.0,5.62,3658715.0,2022-08-26
2022-07-21,769571.0,5.43,4178770.0,2022-08-25
2022-07-20,1045968.0,5.63,5888799.0,2022-08-24
2022-07-19,1532108.0,5.49,8411272.0,2022-08-23
